In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import json
import os

In [2]:
df = pd.read_csv('./data/pm-scraped.tsv', sep='\t')

df.replace('', np.nan, inplace=True)
df.replace('--', np.nan, inplace=True)

In [3]:
#Fix some rows that are misaligned
misaligned = df[df['YOE'].str.startswith('$')]

misaligned_corrected = pd.concat([
    misaligned[['Company', 'Location', 'Date']],
    misaligned.drop(['Company', 'Location', 'Date'], axis=1).shift(axis=1)
], axis=1).fillna('')

df.update(misaligned_corrected)

In [4]:
misaligned = df[df['Base'].str.len() > 5]
misaligned_cols = ['Base', 'Stock', 'Bonus', 'Details', 'Gender']

misaligned_corrected = pd.concat([
    misaligned.drop(misaligned_cols, axis=1),
    misaligned[misaligned_cols].shift(periods=3, axis=1)
], axis=1).fillna('')

df.update(misaligned_corrected)

In [5]:
misaligned = df[~df['Base'].str.contains('^\d', na=True)]
misaligned_cols = ['Base', 'Stock', 'Bonus', 'Details', 'Gender']

misaligned_corrected = pd.concat([
    misaligned.drop(misaligned_cols, axis=1),
    misaligned[misaligned_cols].shift(periods=3, axis=1)
], axis=1).fillna('')

df.update(misaligned_corrected)

In [6]:
misaligned = df[df['Stock'].str.contains('Gender', na=False)]
misaligned_cols = ['Stock', 'Bonus', 'Details', 'Gender']

misaligned_corrected = pd.concat([
    misaligned.drop(misaligned_cols, axis=1),
    misaligned[misaligned_cols].shift(periods=3, axis=1)
], axis=1).fillna('')

df.update(misaligned_corrected)

In [7]:
df.replace('', np.nan, inplace=True)
df['Gender'] = df['Gender'].str.replace('Gender: ', '')
df['Date'] = pd.to_datetime(df['Date'])
df[['yrs_at_company', 'yoe_total']] = pd.DataFrame(df['YOE'].str.split('/').tolist()).astype(float)
df = df.drop(['YOE'], axis=1)

In [8]:
def extract_degree(x):
    if pd.isna(x):
        return np.nan
        
    x = x.lower()
    if 'phd' in x or 'doctor' in x:
        return 'phd'
    elif 'master' in x:
        return 'master'
    elif 'bachelor' in x:
        return 'bachelor'
    else:
        return np.nan

df['degree'] = df['Details'].apply(extract_degree)

In [9]:
df['remote'] = df['Details'].str.lower().str.contains('remote')

In [10]:
#Delete only commas in numbers
df['other'] = df['Details'].str.replace(r'(\d),(\d)', '\\1\\2', regex=True)
df['other'] = df['other'].replace('--', np.nan)
df['other'] = df['other'].str.split(',')

In [11]:
racial_categories = defaultdict(int)

for row in df['other'].tolist():
    if isinstance(row, list):
        for item in row:
            if 'Race:' in item:
                racial_categories[item.split(':')[-1].strip().lower()] += 1
racial_categories

defaultdict(int,
            {'asian': 1423,
             'two or more races': 129,
             'hispanic / latino': 137,
             'white': 1165,
             'black or african american': 101,
             'native hawaiian or other pacific islander': 8,
             'american indian or alaska native': 4})

In [12]:
df = df.drop('other', axis=1)

In [13]:
def extract_racial_category(x):
    if pd.isna(x):
        return np.nan
        
    x = x.lower()

    for k in racial_categories.keys():
        if k in x:
            return k
    return np.nan

df['race'] = df['Details'].apply(extract_racial_category)

In [14]:
def process_num(x):
    if pd.isna(x):
        return np.nan
    if type(x) == float or type(x) == int:
        return x
        
    x = x.lower()
    if 'k' in x:
        if len(x) > 1:
            return float(x.replace('k', '')) * 1000
        return 1000.0
    if 'm' in x:
        if len(x) > 1:
            return float(x.replace('m', '')) * 1000000
        return 1000000.0
    x = float(x)
    return x

In [15]:
df['Base'] = df['Base'].apply(process_num)
df['Stock'] = df['Stock'].apply(process_num)
df['Bonus'] = df['Bonus'].apply(process_num)

In [16]:
df['TC'] = df['TC'].str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype(float)

In [17]:
df.columns = [x.lower() for x in df.columns]

## Geocoding

In [18]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-appldication")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

ModuleNotFoundError: No module named 'geopy'

In [19]:
if os.path.exists('./data/locations.json'):
    with open('./data/locations.json', 'r') as f:
        location_dict = json.load(f)
else:
    locations = df['location'].unique()
    location_dict = {}
    i = 0
    for location in locations:
        print(i, end='\r')
        i+=1
        r = geocode(location, addressdetails=True, language='en')
        if r:
            location_dict[location] = r.raw
        else:
            location_dict[location] = None

    failed = []
    for k, v in location_dict.items():
        if v == None:
            failed.append(k)

    for location in failed:
        location_prefix = ','.join(location.split(',')[:-1])
        print(i, end='\r')
        i+=1
        r = geocode(location_prefix, addressdetails=True, language='en')
        if r:
            location_dict[location] = r.raw
        else:
            location_dict[location] = None

    failed = []
    for k, v in location_dict.items():
        if v == None:
            failed.append(k)

    with open("./data/locations.json", "w") as f:
        json.dump(location_dict, f)

In [20]:
def get_country(x):
    if x and location_dict[x]:
        return location_dict[x]['address']['country']
    return None
df['country'] = df['location'].apply(get_country)

def get_state(x):
    if x and location_dict[x] and 'state' in location_dict[x]['address'].keys():
        return location_dict[x]['address']['state']
    return None
df['state'] = df['location'].apply(get_state)

KeyError: 'New Germany, MN'

In [21]:
df.to_csv('./data/pm-processed.tsv', sep='\t', index=False)